In [1]:
!pip install google-cloud-storage

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
import pandas as pd
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--driver-memory 2g pyspark-shell'
spark = SparkSession.builder.appName('DWTransformation').getOrCreate()

In [3]:
private_key = "ornate-spring-379820-00ec27d088ba.json"
project_id = 'ornate-spring-379820'

gcs_bucket = "bq-bucketgabrielcordeiro-1"
gcs_folder = "transformed_data"
filename = "xxx.parquet"

In [4]:
spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile",private_key)

In [5]:
filename = '01_tabela2.parquet'
filename2 = '02_tabela8.parquet'
filename3 = '03_tabela12.parquet'
df1 = spark.read.parquet(f'gs://{gcs_bucket}/{gcs_folder}/{filename}')
df2 = spark.read.parquet(f'gs://{gcs_bucket}/{gcs_folder}/{filename2}')
df3 = spark.read.parquet(f'gs://{gcs_bucket}/{gcs_folder}/{filename3}')

---

### DIM_Estado

In [6]:
df = df2.select(col("nome_estado"))
window = Window.orderBy('nome_estado')
df = df.withColumn('id_estado', row_number().over(window))

In [7]:
coordinates = { 'Acre': (-9.0238, -70.812),'Alagoas': (-9.5713, -36.7819),'Amapá': (0.902, -52.003),
                'Amazonas': (-3.4168, -65.8561),'Bahia': (-12.5797, -41.7007),'Ceará': (-5.4984, -39.3206),
                'Distrito Federal': (-15.7998, -47.8645),'Espírito Santo': (-19.1834, -40.3089),'Goiás': (-15.827, -49.8362),
                'Maranhão': (-5.4324, -45.4436),'Mato Grosso': (-12.6819, -56.9211),'Mato Grosso do Sul': (-20.7722, -54.7852),
                'Minas Gerais': (-18.5122, -44.555),'Pará': (-3.0144, -52.2024),'Paraíba': (-7.24, -36.782),
                'Paraná': (-24.89, -51.55),'Pernambuco': (-8.8137, -36.9541),'Piauí': (-7.7183, -42.7289),
                'Rio de Janeiro': (-22.84, -43.15),'Rio Grande do Norte': (-5.4026, -36.9541),'Rio Grande do Sul': (-30.0346, -51.2177),
                'Rondônia': (-10.83, -63.34),'Roraima': (2.7376, -62.0751),'Santa Catarina': (-27.2423, -50.2189),
                'São Paulo': (-22.19, -48.79),'Sergipe': (-10.5741, -37.3857),'Tocantins': (-10.25, -48.25) }

In [8]:
df = df.toPandas()

def extract_coordinates(estado_nome):
    return coordinates[estado_nome]

df['latitude'], df['longitude'] = zip(*df['nome_estado'].apply(extract_coordinates))

In [9]:
DIM_estado = spark.createDataFrame(df)

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


### DIM_Municipio

In [10]:
DIM_Municipio = df3.select(col("nome_municipio")).distinct().orderBy("nome_municipio")
DIM_Municipio = DIM_Municipio.repartition('nome_municipio')
window = Window.orderBy('nome_municipio')
DIM_Municipio = DIM_Municipio.withColumn('id_municipio', row_number().over(window))

### DIM_Atividade

In [11]:
DIM_atividade = df3.select(col("classificacao_atividade")).distinct().orderBy("classificacao_atividade")
DIM_atividade = DIM_atividade.filter(~col("classificacao_atividade").contains("Total"))
DIM_atividade = DIM_atividade.repartition("classificacao_atividade")
window2 = Window.orderBy("classificacao_atividade")
DIM_atividade = DIM_atividade.withColumn("id_atividade", row_number().over(window2))

### Stage table

In [12]:
pd.set_option('display.max_columns', False)
pd.set_option('display.max_rows', False)

In [13]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [14]:
stage_df = (df1.join(df2, "UF", how="full_outer")
           .join(df3, "UF", how="full_outer")
           .join(DIM_atividade, "classificacao_atividade", how="full_outer")
           .join(DIM_Municipio, "nome_municipio", how="full_outer")
           .join(DIM_estado, "nome_estado", how="full_outer"))

### FA_Economia

In [15]:
FA_Economia = stage_df.select('id_atividade','id_municipio', 'id_estado',
                              'salario_medio_mensal_valor','pessoal_ocupado_assalariado',
                              'pib_concorrente','tributo_iluminacao_publica','tributo_coleta_de_lixo',
                              'tributo_incendio','tributo_limpeza_publica','poder_da_policia',
                              'outros_tributos','nao_cobra_tributo','posicao_30_maiores_municipios')

FA_Economia = FA_Economia.withColumnRenamed('salario_medio_mensal_valor','salario_medio_mensal')
FA_Economia = FA_Economia.withColumnRenamed('pessoal_ocupado_assalariado','Quant_Pessoal_Assalariado')
FA_Economia = FA_Economia.withColumnRenamed('pib_concorrente','PIB_nominal')
FA_Economia = FA_Economia.withColumnRenamed('tributo_iluminacao_publica','Quant_Muni_Iluminacao_Publica')
FA_Economia = FA_Economia.withColumnRenamed('tributo_coleta_de_lixo','Quant_Muni_Coleta_Lixo')
FA_Economia = FA_Economia.withColumnRenamed('tributo_incendio','Quant_Muni_Trib_Incendio')
FA_Economia = FA_Economia.withColumnRenamed('tributo_limpeza_publica','Quant_Muni_Limp_Publica')
FA_Economia = FA_Economia.withColumnRenamed('poder_da_policia','Quant_Muni_Poder_Policial')
FA_Economia = FA_Economia.withColumnRenamed('outros_tributos','Quant_Muni_Outros_Trib')
FA_Economia = FA_Economia.withColumnRenamed('nao_cobra_tributo','Quant_Muni_Sem_Trib')
FA_Economia = FA_Economia.withColumnRenamed('posicao_30_maiores_municipios','ranking_municipio')

---

In [16]:
FA_Economia.write.mode('overwrite').format('parquet').save(f'gs://{gcs_bucket}/{gcs_folder}/FA_Economia.parquet')
DIM_atividade.write.mode('overwrite').format('parquet').save(f'gs://{gcs_bucket}/{gcs_folder}/DIM_Atividade.parquet')
DIM_Municipio.write.mode('overwrite').format('parquet').save(f'gs://{gcs_bucket}/{gcs_folder}/DIM_Municipio.parquet')
DIM_estado.write.mode('overwrite').format('parquet').save(f'gs://{gcs_bucket}/{gcs_folder}/DIM_Estado.parquet')

NameError: name 'DIM_Estado' is not defined